In [12]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification


In [10]:
import pandas as pd
df=pd.read_csv("patient_queries.csv")
df

,question,predicted_disease
0,I've been dealing with muscle pain for a week ...,Fibromyalgia
1,"Whenever I sitting for long hours, I experienc...",Fibromyalgia
2,Could chills be related to alcohol consumption?,Sepsis
3,"Recently, I noticed rash, and it's getting wor...",Eczema
4,"Whenever I walking, I experience muscle pain. ...",Fibromyalgia
...,...,...
1995,"Whenever I sitting for long hours, I experienc...",Kidney disease
1996,Is it normal to have bloating along with weakn...,IBS
1997,"Whenever I walking, I experience headache. Is ...",Tension headache
1998,Is it normal to have high fever along with nau...,Typhoid fever


In [14]:
df.predicted_disease.unique()

array(['Fibromyalgia', 'Sepsis', 'Eczema', 'Hypertension', 'Dehydration',
       'Retinal detachment', 'Stroke', 'Alzheimers', 'Asthma', 'Gout',
       'IBS', 'Meningitis', 'Rheumatoid arthritis', 'Brain tumor',
       'Influenza', 'Common cold', 'Rosacea', 'Diabetes', 'Brain injury',
       'Chickenpox', 'Epilepsy', 'Neuropathy', 'Anemia', 'Blood clot',
       'Heart attack', 'Gastroenteritis', 'Cancer', 'Malnutrition',
       'Gastritis', 'Arthritis', 'Sinusitis', 'Herpes',
       'Menstrual cramps', 'Hypercalcemia', 'Appendicitis', 'Anaphylaxis',
       'Migraine', 'Fungal infections', 'Malaria', 'Gallstones',
       'Bronchitis', 'Myasthenia gravis', 'Low blood pressure',
       'Irritable Bowel Syndrome IBS', 'Food poisoning', 'Tuberculosis',
       'Heart failure', 'Typhoid fever', 'GERD', 'Angioedema',
       'Motion sickness', 'Depression', 'Hyperthyroidism', 'COPD',
       'Pinched nerve', 'Kidney disease', 'Colon obstruction',
       'Hypothyroidism', 'Lyme disease', 'Burns',

In [35]:
from sklearn.preprocessing import LabelEncoder

X_text = df["question"].astype(str).tolist()
y_label = df["predicted_disease"].astype(str).tolist()

le=LabelEncoder()
y_encoded=le.fit_transform(y_label)

label_mapping=dict(zip(le.classes_,le.fit_transform(le.classes_)))
label_mapping

{'Acid reflux': 0,
 'Allergic reaction': 1,
 'Allergies': 2,
 'Alzheimers': 3,
 'Anaphylaxis': 4,
 'Anemia': 5,
 'Angina': 6,
 'Angioedema': 7,
 'Anxiety': 8,
 'Appendicitis': 9,
 'Arthritis': 10,
 'Asthma': 11,
 'Blood clot': 12,
 'Brain infection encephalitis': 13,
 'Brain injury': 14,
 'Brain tumor': 15,
 'Bronchitis': 16,
 'Burns': 17,
 'COPD': 18,
 'COVID-19': 19,
 'Cancer': 20,
 'Celiac disease': 21,
 'Cellulitis': 22,
 'Chickenpox': 23,
 'Cholera': 24,
 'Chronic Fatigue Syndrome': 25,
 'Colon obstruction': 26,
 'Common cold': 27,
 'Contact dermatitis': 28,
 'Crohns disease': 29,
 'Dehydration': 30,
 'Dementia': 31,
 'Dengue': 32,
 'Depression': 33,
 'Diabetes': 34,
 'Eczema': 35,
 'Epilepsy': 36,
 'Fibromyalgia': 37,
 'Food poisoning': 38,
 'Fungal infections': 39,
 'GERD': 40,
 'Gallstones': 41,
 'Gastritis': 42,
 'Gastroenteritis': 43,
 'Glaucoma': 44,
 'Gout': 45,
 'Heart attack': 46,
 'Heart failure': 47,
 'Hepatitis': 48,
 'Herpes': 49,
 'Hypercalcemia': 50,
 'Hypertension'

In [36]:
num_labels = len(set(y_encoded))
assert num_labels == len(le.classes_), "Mismatch in num_labels and LabelEncoder classes!"

# ✅ Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(r"D:\bert-base-uncased")

# ✅ Tokenize input text
encodings = tokenizer(X_text, truncation=True, padding=True, max_length=128, return_tensors="pt")

# ✅ Convert to tensors
input_ids = encodings["input_ids"]
attention_mask = encodings["attention_mask"]
labels = torch.tensor(y_encoded, dtype=torch.long)


In [37]:
# ✅ Create dataset
dataset = TensorDataset(input_ids, attention_mask, labels)

# ✅ Create DataLoader
batch_size = 16
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# ✅ Load BERT model
model = BertForSequenceClassification.from_pretrained(
    r"D:\bert-base-uncased",
    num_labels=num_labels,  # Ensure correct label count
    ignore_mismatched_sizes=True
)

# ✅ Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)  # Standard learning rate for BERT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at D:\bert-base-uncased and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([100]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([100, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Program Files\Python38\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [39]:
num_epochs = 10  # You can increase this for better performance
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

# ✅ Evaluation
model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in train_loader:  # Replace with test_loader if you have a separate test set
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).cpu().numpy()

        preds.extend(predictions)
        true_labels.extend(labels.cpu().numpy())

# ✅ Compute accuracy
accuracy = accuracy_score(true_labels, preds)
print(f"Accuracy: {accuracy:.4f}")

# ✅ Fix `target_names` mismatch in classification report
print(classification_report(true_labels, preds, target_names=list(le.classes_)[:num_labels]))

Epoch 1, Loss: 1.9317064046859742
Epoch 2, Loss: 1.753250478744507
Epoch 3, Loss: 1.6447682199478149
Epoch 4, Loss: 1.5784369888305665
Epoch 5, Loss: 1.518433472633362
Epoch 6, Loss: 1.4742684831619264
Epoch 7, Loss: 1.426955578804016
Epoch 8, Loss: 1.3907973890304566
Epoch 9, Loss: 1.3352572703361512
Epoch 10, Loss: 1.3058210167884827
Accuracy: 0.5220
                                           precision    recall  f1-score   support

                              Acid reflux       1.00      0.08      0.15        12
                        Allergic reaction       0.41      0.73      0.52        15
                                Allergies       0.56      0.50      0.53        10
                               Alzheimers       0.83      0.36      0.50        28
                              Anaphylaxis       0.50      0.53      0.51        17
                                   Anemia       0.46      0.82      0.59        28
                                   Angina       0.38      0.64 

C:\Program Files\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
num_epochs = 7  # You can increase this for better performance
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

# ✅ Evaluation
model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in train_loader:  # Replace with test_loader if you have a separate test set
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).cpu().numpy()

        preds.extend(predictions)
        true_labels.extend(labels.cpu().numpy())

# ✅ Compute accuracy
accuracy = accuracy_score(true_labels, preds)
print(f"Accuracy: {accuracy:.4f}")

# ✅ Fix `target_names` mismatch in classification report
print(classification_report(true_labels, preds, target_names=list(le.classes_)[:num_labels]))

Epoch 1, Loss: 1.2840368170738221
Epoch 2, Loss: 1.2464171667099
Epoch 3, Loss: 1.2203383316993714
Epoch 4, Loss: 1.1985924940109254
Epoch 5, Loss: 1.1764062695503235
Epoch 6, Loss: 1.142161304473877
Epoch 7, Loss: 1.129553322315216
Accuracy: 0.5805
                                           precision    recall  f1-score   support

                              Acid reflux       0.47      0.75      0.58        12
                        Allergic reaction       0.43      0.67      0.53        15
                                Allergies       1.00      0.40      0.57        10
                               Alzheimers       0.63      0.43      0.51        28
                              Anaphylaxis       0.54      0.76      0.63        17
                                   Anemia       0.47      0.71      0.56        28
                                   Angina       0.47      0.57      0.52        14
                               Angioedema       0.75      0.23      0.35        13
  

C:\Program Files\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
def predict_disease(symptoms_text):
    inputs = tokenizer(symptoms_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # Reverse label mapping (index → disease)
    label_mapping = {v: k for k, v in dict(zip(le.classes_, le.fit_transform(le.classes_))).items()}

    return label_mapping.get(predicted_class, "Unknown Disease")  # Handle missing keys

# Example test
test_symptoms = "I feel very thirsty, urinate frequently, and have blurry vision."
predicted_disease = predict_disease(test_symptoms)
print("Predicted Disease:", predicted_disease)


Predicted Disease: Diabetes


In [47]:
num_epochs = 3  # You can increase this for better performance
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

# ✅ Evaluation
model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in train_loader:  # Replace with test_loader if you have a separate test set
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).cpu().numpy()

        preds.extend(predictions)
        true_labels.extend(labels.cpu().numpy())

# ✅ Compute accuracy
accuracy = accuracy_score(true_labels, preds)
print(f"Accuracy: {accuracy:.4f}")

# ✅ Fix `target_names` mismatch in classification report
print(classification_report(true_labels, preds, target_names=list(le.classes_)[:num_labels]))

Epoch 1, Loss: 1.1058732719421387
Epoch 2, Loss: 1.094217469215393
Epoch 3, Loss: 1.0869721274375916
Accuracy: 0.5800
                                           precision    recall  f1-score   support

                              Acid reflux       0.45      0.83      0.59        12
                        Allergic reaction       0.48      0.73      0.58        15
                                Allergies       0.83      0.50      0.62        10
                               Alzheimers       0.74      0.50      0.60        28
                              Anaphylaxis       0.62      0.59      0.61        17
                                   Anemia       0.62      0.57      0.59        28
                                   Angina       0.75      0.21      0.33        14
                               Angioedema       0.75      0.23      0.35        13
                                  Anxiety       0.70      0.33      0.45        21
                             Appendicitis       0.8

C:\Program Files\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
import torch

# Save model state dictionary
torch.save(model.state_dict(), "disease_prediction_model.pth")

# Save tokenizer
tokenizer.save_pretrained("tokenizer_directory")

print("Model and tokenizer saved successfully!")


Model and tokenizer saved successfully!


In [5]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

config = BertConfig.from_pretrained("bert-base-uncased", num_labels=100) 

model = BertForSequenceClassification(config)

model.load_state_dict(torch.load("disease_prediction_model.pth"))
model.eval() 
tokenizer = BertTokenizer.from_pretrained("tokenizer_directory")

print("Model and tokenizer loaded successfully!")

Model and tokenizer loaded successfully!


In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
all_diseases = ['Fibromyalgia', 'Sepsis', 'Eczema', 'Hypertension', 'Dehydration',
       'Retinal detachment', 'Stroke', 'Alzheimers', 'Asthma', 'Gout',
       'IBS', 'Meningitis', 'Rheumatoid arthritis', 'Brain tumor',
       'Influenza', 'Common cold', 'Rosacea', 'Diabetes', 'Brain injury',
       'Chickenpox', 'Epilepsy', 'Neuropathy', 'Anemia', 'Blood clot',
       'Heart attack', 'Gastroenteritis', 'Cancer', 'Malnutrition',
       'Gastritis', 'Arthritis', 'Sinusitis', 'Herpes',
       'Menstrual cramps', 'Hypercalcemia', 'Appendicitis', 'Anaphylaxis',
       'Migraine', 'Fungal infections', 'Malaria', 'Gallstones',
       'Bronchitis', 'Myasthenia gravis', 'Low blood pressure',
       'Irritable Bowel Syndrome IBS', 'Food poisoning', 'Tuberculosis',
       'Heart failure', 'Typhoid fever', 'GERD', 'Angioedema',
       'Motion sickness', 'Depression', 'Hyperthyroidism', 'COPD',
       'Pinched nerve', 'Kidney disease', 'Colon obstruction',
       'Hypothyroidism', 'Lyme disease', 'Burns', 'Allergic reaction',
       'Crohns disease', 'Tension headache', 'Restless leg syndrome',
       'COVID-19', 'Psoriasis', 'Brain infection encephalitis',
       'Pulmonary embolism', 'Viral infection flu',
       'Vitamin B12 deficiency', 'Hypoglycemia', 'Contact dermatitis',
       'Dementia', 'Pneumonia', 'Cholera', 'Dengue', 'Celiac disease',
       'Sunburn', 'Cellulitis', 'Polymyositis', 'Glaucoma', 'Menopause',
       'Angina', 'Vertigo', 'Anxiety', 'Peripheral neuropathy',
       'Sleep apnea', 'Peptic ulcer', 'Hepatitis', 'Lupus', 'Allergies',
       'Acid reflux', 'Viral fever', 'Chronic Fatigue Syndrome',
       'Multiple sclerosis', 'Pulmonary edema',
       'Small intestine bacterial overgrowth SIBO', 'Pregnancy',
       'Scabies', 'Lactose intolerance']
le.fit(all_diseases)

print("Model and tokenizer loaded successfully!")

def predict_disease(symptoms_text):
    inputs = tokenizer(symptoms_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    label_mapping = {v: k for k, v in dict(zip(le.classes_, le.fit_transform(le.classes_))).items()}

    return label_mapping.get(predicted_class, "Unknown Disease") 

test_symptoms = "I feel very thirsty, urinate frequently, and have blurry vision."
predicted_disease = predict_disease(test_symptoms)
print("Predicted Disease:", predicted_disease)

Model and tokenizer loaded successfully!
Predicted Disease: Diabetes


In [20]:
predict_disease("I have been experiencing a persistent cough, wheezing, and shortness of breath for the past few weeks. What could be causing these symptoms")

'Asthma'

In [26]:
predict_disease("I am transgender")

'Anxiety'